In [1]:
import numpy as np
import random
import os
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import train_test_split
from sklearn.utils import shuffle
from sklearn.metrics import accuracy_score
import pandas as pd
from sklearn.preprocessing import OneHotEncoder
import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Activation, Flatten, Dense, Dropout, SimpleRNN
from tensorflow.keras.optimizers import SGD
from tensorflow.keras import backend as K
from sklearn.metrics import confusion_matrix, classification_report

C:\Users\Ethan\AppData\Roaming\Python\Python39\site-packages\scipy\__init__.py:146: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.26.0
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion}"


In [2]:
#binarize the labels
lb = LabelBinarizer()

import pandas as pd

data_files = ['cle_train.csv','cle_test.csv','hun_train.csv','hun_test.csv','swi_train.csv','swi_test.csv','vir_train.csv','vir_test.csv']

datasets = {}

for file in data_files:
    data = pd.read_csv('../TrainTestData/' + file)
    
    X = data.iloc[:, :-1]
    Y = data.iloc[:, -1]
    
    Y_binary = Y.apply(lambda x: 1 if x > 0 else 0)
    
    # Extract the name from the file path
    name = file.split('.')[0]
    
    # Store the dataset components in a dictionary
    datasets[name] = {'X': X, 'Y': Y, 'Y_binary': Y_binary}

# Unpack the dictionary values in a loop
variables = ['cle', 'hun', 'swi', 'vir']
train_test = ['train', 'test']

for var in variables:
    for tt in train_test:
        X, Y, Y_binary = datasets[f'{var}_{tt}'].values()
        globals()[f'{var}_X_{tt}'] = X
        globals()[f'{var}_Y_{tt}'] = Y
        globals()[f'{var}_Y_{tt}_binary'] = Y_binary

# creating train test

In [3]:
X_test = pd.concat([cle_X_test,hun_X_test,swi_X_test,vir_X_test])
y_test = pd.concat([cle_Y_test_binary,hun_Y_test_binary,swi_Y_test_binary,vir_Y_test_binary])

X_train = pd.concat([cle_X_train,hun_X_train,swi_X_train,vir_X_train])
y_train = pd.concat([cle_Y_train_binary,hun_Y_train_binary,swi_Y_train_binary,vir_Y_train_binary])

In [4]:
def create_clients():
    cle_zip = list(zip(cle_X_train.values,cle_Y_train_binary))
    hun_zip = list(zip(hun_X_train.values,hun_Y_train_binary))
    vir_zip = list(zip(vir_X_train.values,vir_Y_train_binary))
    swi_zip = list(zip(swi_X_train.values,swi_Y_train_binary))
    
    shards = [cle_zip, hun_zip, vir_zip,swi_zip]
    client_names = ["client_1","client_2","client_3","client_4"]
    dic = {client_names[i] : shards[i] for i in range(len(client_names))}
    return dic


def batch_data(data_shard, bs=32):
    '''Takes in a clients data shard and create a tfds object off it
    args:
        shard: a data, label constituting a client's data shard
        bs:batch size
    return:
        tfds object'''
    #seperate shard into data and labels lists
    data, label = zip(*data_shard)
    dataset = tf.data.Dataset.from_tensor_slices((list(data), list(label)))
    return dataset.shuffle(len(label)).batch(bs)


class DNN:
    @staticmethod
    def build():
        model = Sequential()
        model.add(Dense(64, input_shape=(35,), activation='relu'))
        model.add(Dense(192, activation='relu'))
        model.add(Dense(2, activation='sigmoid'))

        model.compile(loss='sparse_categorical_crossentropy', 
                      optimizer=keras.optimizers.Adam(learning_rate=0.001), 
                      metrics=['accuracy'])
        return model
    

def weight_scalling_factor(clients_trn_data, client_name):
    client_names = list(clients_trn_data.keys())
    #get the bs
    bs = list(clients_trn_data[client_name])[0][0].shape[0]
    #first calculate the total training data points across clinets
    global_count = sum([tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy() for client_name in client_names])*bs
    # get the total number of data points held by a client
    local_count = tf.data.experimental.cardinality(clients_trn_data[client_name]).numpy()*bs
    return local_count/global_count


def scale_model_weights(weight, scalar):
    '''function for scaling a models weights'''
    weight_final = []
    steps = len(weight)
    for i in range(steps):
        weight_final.append(scalar * weight[i])
    return weight_final


def sum_scaled_weights(scaled_weight_list):
    '''Return the sum of the listed scaled weights. The is equivalent to scaled avg of the weights'''
    avg_grad = list()
    #get the average grad accross all client gradients
    for grad_list_tuple in zip(*scaled_weight_list):
        layer_mean = tf.math.reduce_sum(grad_list_tuple, axis=0)
        avg_grad.append(layer_mean)
        
    return avg_grad


def test_model(X_test, Y_test,  model, comm_round):
    cce = tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True)
    logits = model.predict(X_test)
    length = len(y_test)
    Y_test = tf.reshape(Y_test,(length,1))
    loss = cce(Y_test, logits)
    acc = accuracy_score(tf.argmax(logits, axis=1), Y_test)
    cm_cle = confusion_matrix(tf.argmax(logits, axis=1), Y_test)
    recall = cm_cle[1][1]/(cm_cle[1][1]+cm_cle[0][1])
    precision = cm_cle[1][1]/(cm_cle[1][1]+cm_cle[1][0])
    print('comm_round: {} | global_acc: {:.3%} | global_loss: {} | recall_1: {:.4%} | precision_1: {:.4%}'
          .format(comm_round, acc, loss,recall,precision))
    return acc, loss

In [5]:
# create clients
clients = create_clients()

#process and batch the training data for each client
clients_batched = dict()
for (client_name, data) in clients.items():
    clients_batched[client_name] = batch_data(data)
    
#process and batch the test set  
test_batched = tf.data.Dataset.from_tensor_slices((X_test, y_test)).batch(len(y_test))

comms_round = 100

#initialize global model
smlp_global = DNN()
global_model = smlp_global.build()

losslist = []
#commence global training loop
for comm_round in range(comms_round):
            
    # get the global model's weights - will serve as the initial weights for all local models
    global_weights = global_model.get_weights()
    
    #initial list to collect local model weights after scalling
    scaled_local_weight_list = list()

    #randomize client data - using keys
    client_names= list(clients_batched.keys())
    random.shuffle(client_names)
    
    #loop through each client and create new local model
    for client in client_names:
        smlp_local = DNN()
        local_model = smlp_local.build()
        local_model.compile(loss='sparse_categorical_crossentropy', 
                      optimizer=keras.optimizers.Adam(learning_rate=0.001), 
                      metrics=['accuracy'])
        
        #set local model weight to the weight of the global model
        local_model.set_weights(global_weights)
        
        #fit local model with client's data
        with tf.device('/device:GPU:0'):
            local_model.fit(clients_batched[client], epochs=1, verbose=0)
        
        #scale the model weights and add to list
        scaling_factor = weight_scalling_factor(clients_batched, client)
        scaled_weights = scale_model_weights(local_model.get_weights(), scaling_factor)
        scaled_local_weight_list.append(scaled_weights)
        
        #clear session to free memory after each communication round
        K.clear_session()
        
    #to get the average over all the local model, we simply take the sum of the scaled weights
    average_weights = sum_scaled_weights(scaled_local_weight_list)
    
    #update global model 
    global_model.set_weights(average_weights)

    #test global model and print out metrics after each communications round
    for(X_test, Y_test) in test_batched:
        global_acc, global_loss = test_model(X_test, Y_test, global_model, comm_round)
        losslist.append(global_loss)

    global_model.save('../Models/FL_DNN/train/FL_DNN_epoch'+str(comm_round)+'.h5')

1773/1773 [==============================] - 2s 955us/step
comm_round: 0 | global_acc: 76.100% | global_loss: 0.6062946915626526 | recall_1: 59.4459% | precision_1: 75.1558%
1773/1773 [==============================] - 2s 1ms/step
comm_round: 1 | global_acc: 76.220% | global_loss: 0.6042347550392151 | recall_1: 55.1303% | precision_1: 78.5737%
1773/1773 [==============================] - 2s 1ms/step
comm_round: 2 | global_acc: 76.373% | global_loss: 0.601868748664856 | recall_1: 57.2082% | precision_1: 77.3781%
1773/1773 [==============================] - 2s 981us/step
comm_round: 3 | global_acc: 76.345% | global_loss: 0.6021053791046143 | recall_1: 56.1293% | precision_1: 78.1238%
1773/1773 [==============================] - 2s 981us/step
comm_round: 4 | global_acc: 76.355% | global_loss: 0.5989999175071716 | recall_1: 56.8752% | precision_1: 77.5799%
1773/1773 [==============================] - 2s 996us/step
comm_round: 5 | global_acc: 76.340% | global_loss: 0.5977829098701477 | reca

1773/1773 [==============================] - 2s 962us/step
comm_round: 47 | global_acc: 76.459% | global_loss: 0.6009507179260254 | recall_1: 58.6556% | precision_1: 76.5633%
1773/1773 [==============================] - 2s 970us/step
comm_round: 48 | global_acc: 76.396% | global_loss: 0.599286675453186 | recall_1: 59.0596% | precision_1: 76.1289%
1773/1773 [==============================] - 2s 1ms/step
comm_round: 49 | global_acc: 76.373% | global_loss: 0.5973886251449585 | recall_1: 58.7000% | precision_1: 76.3161%
1773/1773 [==============================] - 2s 939us/step
comm_round: 50 | global_acc: 76.318% | global_loss: 0.5977978110313416 | recall_1: 57.8564% | precision_1: 76.7658%
1773/1773 [==============================] - 2s 970us/step
comm_round: 51 | global_acc: 76.415% | global_loss: 0.6013373136520386 | recall_1: 59.8721% | precision_1: 75.6394%
1773/1773 [==============================] - 2s 977us/step
comm_round: 52 | global_acc: 76.392% | global_loss: 0.597463071346283

1773/1773 [==============================] - 2s 1ms/step
comm_round: 94 | global_acc: 76.228% | global_loss: 0.5976687669754028 | recall_1: 58.1050% | precision_1: 76.3625%
1773/1773 [==============================] - 2s 988us/step
comm_round: 95 | global_acc: 76.241% | global_loss: 0.6015323400497437 | recall_1: 58.9841% | precision_1: 75.7973%
1773/1773 [==============================] - 2s 967us/step
comm_round: 96 | global_acc: 76.266% | global_loss: 0.6010911464691162 | recall_1: 58.8687% | precision_1: 75.9349%
1773/1773 [==============================] - 2s 1ms/step
comm_round: 97 | global_acc: 76.213% | global_loss: 0.6006035804748535 | recall_1: 57.7898% | precision_1: 76.5422%
1773/1773 [==============================] - 2s 984us/step
comm_round: 98 | global_acc: 76.216% | global_loss: 0.5992223024368286 | recall_1: 57.4613% | precision_1: 76.7843%
1773/1773 [==============================] - 2s 963us/step
comm_round: 99 | global_acc: 76.186% | global_loss: 0.5966618657112122

# Testing on each dataset on the best performanced model

In [6]:
global_model = tf.keras.models.load_model('../Models/FL_DNN/train/FL_DNN_epoch30.h5')

In [7]:
Y_predictions = np.argmax(global_model.predict(X_test),axis = 1)
cm = confusion_matrix(Y_predictions, Y_test)
print(cm)
print(classification_report(Y_test, Y_predictions, digits=4))

1773/1773 [==============================] - 2s 1ms/step
[[29604  8820]
 [ 4609 13703]]
              precision    recall  f1-score   support

           0     0.7705    0.8653    0.8151     34213
           1     0.7483    0.6084    0.6711     22523

    accuracy                         0.7633     56736
   macro avg     0.7594    0.7368    0.7431     56736
weighted avg     0.7617    0.7633    0.7580     56736



In [8]:
Y_cle = np.argmax(global_model.predict(cle_X_test),axis = 1)
cm_cle = confusion_matrix(Y_cle, cle_Y_test_binary)
print(np.bincount(cle_Y_test_binary))
print(cm_cle)
recall = cm_cle[1][1]/np.bincount(cle_Y_test_binary)[1]
print('actual recall for class 1 is: ' + str(recall))
print(classification_report(Y_cle, cle_Y_test_binary, digits=4))

444/444 [==============================] - 0s 1ms/step
[8535 5649]
[[7365 2214]
 [1170 3435]]
actual recall for class 1 is: 0.608072225172597
              precision    recall  f1-score   support

           0     0.8629    0.7689    0.8132      9579
           1     0.6081    0.7459    0.6700      4605

    accuracy                         0.7614     14184
   macro avg     0.7355    0.7574    0.7416     14184
weighted avg     0.7802    0.7614    0.7667     14184



In [9]:
Y_vir = np.argmax(global_model.predict(vir_X_test),axis = 1)
cm_vir = confusion_matrix(Y_vir, vir_Y_test_binary)
print(np.bincount(vir_Y_test_binary))
print(cm_vir)
recall = cm_vir[1][1]/np.bincount(vir_Y_test_binary)[1]
print('actual recall for class 1 is: ' + str(recall))
print(classification_report(Y_vir, vir_Y_test_binary, digits=4))

444/444 [==============================] - 0s 1ms/step
[8647 5537]
[[7495 2165]
 [1152 3372]]
actual recall for class 1 is: 0.6089940400939137
              precision    recall  f1-score   support

           0     0.8668    0.7759    0.8188      9660
           1     0.6090    0.7454    0.6703      4524

    accuracy                         0.7661     14184
   macro avg     0.7379    0.7606    0.7446     14184
weighted avg     0.7846    0.7661    0.7714     14184



In [10]:
Y_hun = np.argmax(global_model.predict(hun_X_test),axis = 1)
cm_hun = confusion_matrix(Y_hun, hun_Y_test_binary)
print(np.bincount(hun_Y_test_binary))
print(cm_hun)
recall = cm_hun[1][1]/np.bincount(hun_Y_test_binary)[1]
print('actual recall for class 1 is: ' + str(recall))
print(classification_report(Y_hun, hun_Y_test_binary, digits=4))

444/444 [==============================] - 0s 1ms/step
[8541 5643]
[[7419 2198]
 [1122 3445]]
actual recall for class 1 is: 0.6104908736487684
              precision    recall  f1-score   support

           0     0.8686    0.7714    0.8172      9617
           1     0.6105    0.7543    0.6748      4567

    accuracy                         0.7659     14184
   macro avg     0.7396    0.7629    0.7460     14184
weighted avg     0.7855    0.7659    0.7713     14184



In [11]:
Y_swi = np.argmax(global_model.predict(swi_X_test),axis = 1)
cm_swi = confusion_matrix(Y_swi, swi_Y_test_binary)
print(np.bincount(swi_Y_test_binary))
print(cm_swi)
recall = cm_swi[1][1]/np.bincount(swi_Y_test_binary)[1]
print('actual recall for class 1 is: ' + str(recall))
print(classification_report(Y_swi, swi_Y_test_binary, digits=4))

444/444 [==============================] - 0s 1ms/step
[8490 5694]
[[7325 2243]
 [1165 3451]]
actual recall for class 1 is: 0.6060765718299965
              precision    recall  f1-score   support

           0     0.8628    0.7656    0.8113      9568
           1     0.6061    0.7476    0.6694      4616

    accuracy                         0.7597     14184
   macro avg     0.7344    0.7566    0.7404     14184
weighted avg     0.7792    0.7597    0.7651     14184

